In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

appliances_dataframe = getDF("Pet_Supplies_5.json.gz')
appliances_dataframe.to_pickle("pet_supplies_amazon_review_data.pkl")

# Code to create reviews in a folder from source file. 
Function that when called and used the proper parameters will read in the .pkl file and create the .txt review inside the specified directory, and create it under the desired name.

In [8]:
import pandas as pd
import re

def create_review_in_folder(review_list,folder_name,file_name_cat):
    number = 1
    reviews = pd.read_pickle(review_list)
    reviews = reviews['reviewText'].tolist()
    reviews = list(set(reviews))
    for review in reviews:
        review_len = str(review).split()
        if len(review_len) >= 30:
            path = "data/" + folder_name
            file_name = file_name_cat  + str(number) + ".txt"      
            number += 1
            
            document = re.sub(r'\W', ' ', str(review))
            document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
            document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
            document = re.sub(r'\s+', ' ', document, flags=re.I)
            document = re.sub(r'^b\s+', '', document)
            document = document.lower()
            document = document.split()
            document = ' '.join(document)

            path = path + file_name
            file = open(path,"x")
            file.write(str(document))
            file.close()         
        else:      
            continue      
    return


    


# Sample useage of the create_review_in_folder function.
Whenever you pass in the .pkl file, the directory, and the desired name for the review files it will read the pkl file and create in directory as requested.

In [11]:
create_review_in_folder("video_game_amazon_review.pkl","Video Games Amazon Reviews/","video_games_amazon_review_")

C:\Users\Isaac\Documents\SCHOOL\CPSC4430\Final Project\Text-Classification-of-Amazon-Review-Data


# Section I.
Located cell below is used for simple logistic regression using a Count Vectorizer to embedd each word as an instance. We achieve a 0.9376 % accuracy for our model with no other modifications. 

In [1]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)


classifier = LogisticRegression()
classifier.fit(X_train_data, y_train)
score = classifier.score(X_test_data, y_test)

print("Accuracy:", score)

Accuracy: 0.9376


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Section I. a
Located below is the change from the default lbfgs optimizer to the newton-cg optimizer which gives us a marginal increase to 0.9382%, a change from the previous accuracy by 0.005%! Wow!

In [13]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)


classifier = LogisticRegression(solver = "newton-cg") 
classifier.fit(X_train_data, y_train)
score = classifier.score(X_test_data, y_test)

print("Accuracy:", score)

Accuracy: 0.9382


# Section I. B
Located section below is the code to produce multiple logistic regression models with different maximum iterations where the best performing model had the maximum iterations set at and had the accuracy of.

In [11]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)


#Creating array of values to test for maximum iterations on logistic regression model. Iterations are increments of 50, the default is 100. 
maximum_iterations_array = [100,150,200,250,300]

for iteration in maximum_iterations_array:
    classifier = LogisticRegression(max_iter=iteration) 
    classifier.fit(X_train_data, y_train)
    score = classifier.score(X_test_data, y_test)
    print("Accuracy for max_iter = " + str(iteration) + " : " + str(score))

C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for max_iter = 100 : 0.9377


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for max_iter = 150 : 0.9379


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for max_iter = 200 : 0.9382


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy for max_iter = 250 : 0.9386
Accuracy for max_iter = 300 : 0.9384


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Section I. C
Located cell below is used for setting the maximum iteration of our logistic regression model to 250, and setting the optimizer function to use the Newton-CG method. Our reported accuracy from this model was: 

In [1]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)

#Creating the logistic regression model at the calculated max iterations of 250 using the newton-cg optimizer.
classifier = LogisticRegression(max_iter=250,solver = "newton-cg") 
classifier.fit(X_train_data, y_train)
score = classifier.score(X_test_data, y_test)

print("Accuracy:", score)

Accuracy: 0.9382


# Section I. D
Located cell below is used for setting the multi_class hyerparameter to change the loss functionality. 

In [2]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)

#Creating the logistic regression model at the calculated max iterations of 250 using the newton-cg optimizer.
classifier = LogisticRegression(max_iter=250) 
classifier.fit(X_train_data, y_train)
score = classifier.score(X_test_data, y_test)
print("Default Accuracy:", score)

ovr_classifier = LogisticRegression(max_iter=250,multi_class="ovr") 
ovr_classifier.fit(X_train_data, y_train)
ovr_score = ovr_classifier.score(X_test_data, y_test)
print("Ovr Accuracy:", ovr_score)

multi_classifier = LogisticRegression(max_iter=250,multi_class="ovr") 
multi_classifier.fit(X_train_data, y_train)
multi_score = multi_classifier.score(X_test_data, y_test)
print("Multi Accuracy:", multi_score)


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Default Accuracy: 0.9386


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Ovr Accuracy: 0.9433


C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Isaac\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Ovr Accuracy: 0.9433


# Section II.
Located cell below is used for a simple Random Forest Classifier with the default N-estimators set at 100, with no random state. We achieve an accuracy of: 0.8951 for this base model. 

In [10]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)

#Creating the random forest classifier with the default n_estimators to 100 and no random state. 
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train_data, y_train)

forrest_prediction = classifier.predict(X_test_data)
print(classification_report(y_test,forrest_prediction))
print(accuracy_score(y_test, forrest_prediction))


0.881


# Section II. a
Located cell below is the change from the criterion for the classifer from the default of Gini-Index. The model reports an accuracy of 0.881. A minimal difference of 0.0151% for using Entropy.

In [12]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)

#Changed classifer hyper-parameter from the default Gini-Index to Entropy to determine the split. 
classifier = RandomForestClassifier(n_estimators=100, random_state=0,criterion="entropy")
classifier.fit(X_train_data, y_train)

forrest_prediction = classifier.predict(X_test_data)
print(classification_report(y_test,forrest_prediction))
print(accuracy_score(y_test, forrest_prediction))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1974
           1       0.85      0.83      0.84      1945
           2       0.90      0.86      0.88      2081
           3       0.81      0.82      0.81      2029
           4       0.94      0.95      0.94      1971

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

0.881


# Section II. B
Located cell below is used to loop through an array of N-Estimators to print out and see the various accuracy for each N-Number of Trees. 

In [4]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

review_data = load_files(r"data/")
X, y = review_data.data , review_data.target

review_data_train, review_data_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(review_data_train)


X_train_data = vectorizer.transform(review_data_train)
X_test_data = vectorizer.transform(review_data_test)

n_estimators_array = [100,120,140,160,180,200]

for n_estimator in n_estimators_array:
    classifier = RandomForestClassifier(n_estimators=n_estimator, random_state=0)
    classifier.fit(X_train_data, y_train)
    forrest_prediction = classifier.predict(X_test_data)
    print("Accuracy for N-Trees = " + str(n_estimator) + " : " + str(accuracy_score(y_test, forrest_prediction)))




Accuracy for max_iter = 100 : 0.8961
Accuracy for max_iter = 120 : 0.8982
Accuracy for max_iter = 140 : 0.8983
Accuracy for max_iter = 160 : 0.8989
Accuracy for max_iter = 180 : 0.9029
Accuracy for max_iter = 200 : 0.9018
